In [ ]:
try:
  %load_ext tensorboard
except:
  pass

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
import tensorflow as tf
from PIL import Image
import glob
import matplotlib.pyplot as plt
import cv2
from sklearn.metrics import accuracy_score,confusion_matrix
from statistics import median

In [ ]:
class EarlyStoppingCallback(keras.callbacks.Callback):
  def __init__(self,patience=0):
    super(EarlyStoppingCallback,self).__init__()
    self.patience=patience
    self.writer=tf.summary.create_file_writer('./logs')
  def on_train_begin(self,logs=None):
    self.best=np.Inf
    self.wait=0#for countinf waiting epoch
    self.stopped_epoch=0
  def on_epoch_end(self,epoch,logs=None):
    current_loss=logs.get("val_loss")
    if np.less(current_loss,self.best):
      self.best=current_loss
      self.wait=0
      self.best_weights=self.model.get_weights()
    else:
      self.wait+=1
      print("\nwait mode, step: %d"% self.wait)
      if self.wait>=self.patience:
        self.stopped_epoch=epoch
        self.model.stop_training=True#########################training vaymise
        self.model.set_weights(self.best_weights)
      with self.writer.as_default():
        tf.summary.scalar('wait',self.wait,step=epoch)#mitonim jaye summary , histogram ,image ya harja ke mikhaym chizi benevisim bezarim
      self.writer.flush()
  #check if we stop or early forc to stop
  def on_train_end(self,logs=None):
    if self.stopped_epoch>0:
      print("epoch: %d : early stopping."% self.stopped_epoch)


In [ ]:
def load_file():
  outdoor_path='/content/drive/MyDrive/Colab Notebooks/NN/hw4/dataset/outdoor/*'#global path for outdoor
  indoor_path='/content/drive/MyDrive/Colab Notebooks/NN/hw4/dataset/indoor/*'#global path for indoor
  img=[]#list of images in real size
  label=[]#all label
  #reading file and fill img in lable
  for filename in glob.glob(outdoor_path):
    img.append(np.array(Image.open(filename)))
    label.append(0)
  for filename in glob.glob(indoor_path):
    img.append(np.array(Image.open(filename)))
    label.append(1)
  img=np.array(img)#change list to numpy array
  label=np.array(label)#change to numpy array
  x_shape=[img[i].shape[0] for i in range(len(img))]#list of x shape of images
  y_shape=[img[i].shape[1] for i in range(len(img))]#list of y shape of images
  median_x=int(median(x_shape))-150#find median in x cordinate and minus 150 (beacuse I recive memory error with median in fiting network)
  median_y=int(median(y_shape))-150#find median in x cordinate and minus 150 
  print('x_shape and y_shape of one images')
  print(median_x,median_y)
  resize_img=[]#resize images and save that in this list
  for k in range(len(img)):
    resize_img.append(np.array(cv2.resize(img[k],(median_y,median_x))))
  resize_img=np.asarray(resize_img)
  return resize_img,label,median_x,median_y

In [ ]:
img,label,shape_x,shape_y=load_file()#list of images and their label

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  del sys.path[0]


x_shape and y_shape of one images
257 316


In [ ]:
img.shape

(800, 257, 316, 3)

In [ ]:
#spliting train and test and valid test
#X_testing_data,_,y_testing_data,_=train_test_split(img, label, test_size=0.35, random_state=42)
#X_train, X_test, y_train, y_test = train_test_split(X_testing_data, y_testing_data, test_size=0.3, random_state=42)
#X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.33, random_state=42)
X_train, X_remain, y_train, y_remain = train_test_split(img, label, test_size=0.3, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_remain, y_remain, test_size=0.33, random_state=42)

In [ ]:
len(X_train)

364

#ResNet

In [ ]:
res_based_model=keras.applications.ResNet152(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(shape_x,shape_y,3),
    pooling=None,
)
res_based_model.trainable=False

In [ ]:
model_res=keras.models.Sequential()
model_res.add(res_based_model)
model_res.add(keras.layers.Flatten())
model_res.add(keras.layers.Dense(512,activation='relu'))
model_res.add(keras.layers.Dense(2,activation='softmax'))
layer=model_res.layers[0]
layer.trainable=False

In [ ]:
%rm -rf './logs'
tb_callback=keras.callbacks.TensorBoard(log_dir="./logs",histogram_freq=1)
model_res.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss=keras.losses.SparseCategoricalCrossentropy(True),
              metrics=['accuracy'])
es_callback=EarlyStoppingCallback(patience=5)
model_res.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=300, callbacks=(tb_callback,es_callback))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 1/300


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:4930: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


18/18 [==============================] - 23s 849ms/step - loss: 1.6074 - accuracy: 0.8554 - val_loss: 0.1605 - val_accuracy: 0.9750
Epoch 2/300
18/18 [==============================] - 10s 564ms/step - loss: 0.1164 - accuracy: 0.9696 - val_loss: 0.1105 - val_accuracy: 0.9688
Epoch 3/300
18/18 [==============================] - 10s 562ms/step - loss: 0.0151 - accuracy: 0.9946 - val_loss: 0.4709 - val_accuracy: 0.9563

wait mode, step: 1
Epoch 4/300
18/18 [==============================] - 10s 551ms/step - loss: 0.0066 - accuracy: 0.9982 - val_loss: 0.2659 - val_accuracy: 0.9750

wait mode, step: 2
Epoch 5/300
18/18 [==============================] - 10s 547ms/step - loss: 4.5016e-04 - accuracy: 1.0000 - val_loss: 0.2323 - val_accuracy: 0.9750

wait mode, step: 3
Epoch 6/300
18/18 [==============================] - 10s 568ms/step - loss: 1.7832e-05 - accuracy: 1.0000 - val_loss: 0.2338 - val_accuracy: 0.9750

wait mode, step: 4
Epoch 7/300
18/18 [==============================] - 10s 539

In [ ]:
pred=np.argmax(model_res.predict(X_test),axis=1)
print('accuracy of test for model with fix resnet weight==>%.4f'% accuracy_score(y_test,pred))
print('confusion matrix of test for model with fix resnet weight==>\n', confusion_matrix(y_test,pred))

accuracy of test for model with fix resnet weight==>0.9625
confusion matrix of test for model with fix resnet weight==>
 [[42  0]
 [ 3 35]]


#fine tune in feature part

In [ ]:
for layer in res_based_model.layers[:-50]:
    layer.trainable = False
for layer in res_based_model.layers[-50:]:
    layer.trainable = True


In [ ]:
model_res_2=keras.models.Sequential()
model_res_2.add(res_based_model)
for i in range(1,len(model_res.layers)):
  model_res_2.add(model_res.layers[i])

In [ ]:
model_res_2.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss=keras.losses.SparseCategoricalCrossentropy(True),
              metrics=['accuracy'])

In [ ]:
%rm -rf './logs'
tb_callback=keras.callbacks.TensorBoard(log_dir="./logs",histogram_freq=1)

In [ ]:
es_callback=EarlyStoppingCallback(patience=5)
model_res_2.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=300, callbacks=(tb_callback,es_callback))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 1/300


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:4930: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


18/18 [==============================] - 22s 809ms/step - loss: 0.2509 - accuracy: 0.9661 - val_loss: 0.9593 - val_accuracy: 0.9312
Epoch 2/300
18/18 [==============================] - 10s 547ms/step - loss: 0.0465 - accuracy: 0.9875 - val_loss: 0.6156 - val_accuracy: 0.9500
Epoch 3/300
18/18 [==============================] - 10s 570ms/step - loss: 0.0110 - accuracy: 0.9982 - val_loss: 0.5576 - val_accuracy: 0.9750
Epoch 4/300
18/18 [==============================] - 10s 572ms/step - loss: 1.7221e-07 - accuracy: 1.0000 - val_loss: 0.6126 - val_accuracy: 0.9625

wait mode, step: 1
Epoch 5/300
18/18 [==============================] - 10s 551ms/step - loss: 2.0137e-07 - accuracy: 1.0000 - val_loss: 0.5706 - val_accuracy: 0.9625

wait mode, step: 2
Epoch 6/300
18/18 [==============================] - 10s 572ms/step - loss: 1.1580e-07 - accuracy: 1.0000 - val_loss: 0.5252 - val_accuracy: 0.9625
Epoch 7/300
18/18 [==============================] - 10s 571ms/step - loss: 2.2565e-08 - accurac

In [ ]:
%tensorboard --logdir "./logs"

In [ ]:
pred=np.argmax(model_res_2.predict(X_test),axis=1)
print('accuracy of test for model with finetuen resnet weight==>%.4f'% accuracy_score(y_test,pred))
print('confusion matrix of test for model with finetuen resnet weight==>\n', confusion_matrix(y_test,pred))

accuracy of test for model with finetuen resnet weight==>0.9750
confusion matrix of test for model with finetuen resnet weight==>
 [[42  0]
 [ 2 36]]


#DenseNet

In [ ]:
dense_based_model=keras.applications.DenseNet201(
    include_top=False,
    weights="imagenet",
    input_shape=(shape_x,shape_y,3),
    pooling=None,
)
dense_based_model.trainable=False

In [ ]:
model_dense=keras.models.Sequential()
model_dense.add(dense_based_model)
model_dense.add(keras.layers.Flatten())
model_res.add(keras.layers.Dense(256,activation='relu'))
model_res.add(keras.layers.Dense(128,activation='relu'))
model_dense.add(keras.layers.Dense(2,activation='softmax'))
layer=model_res.layers[0]
layer.trainable=False

In [ ]:
%rm -rf './logs'
tb_callback=keras.callbacks.TensorBoard(log_dir="./logs",histogram_freq=1)

In [ ]:
model_dense.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss=keras.losses.SparseCategoricalCrossentropy(True),
              metrics=['accuracy'])
es_callback=EarlyStoppingCallback(patience=5)

In [ ]:
model_dense.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=300, callbacks=(tb_callback,es_callback))

Epoch 1/300


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:4930: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


18/18 [==============================] - 23s 778ms/step - loss: 1.7675 - accuracy: 0.7179 - val_loss: 0.4254 - val_accuracy: 0.9000
Epoch 2/300
18/18 [==============================] - 5s 299ms/step - loss: 0.4529 - accuracy: 0.9089 - val_loss: 0.9141 - val_accuracy: 0.8625

wait mode, step: 1
Epoch 3/300
18/18 [==============================] - 5s 297ms/step - loss: 0.2449 - accuracy: 0.9321 - val_loss: 0.9411 - val_accuracy: 0.8875

wait mode, step: 2
Epoch 4/300
18/18 [==============================] - 5s 293ms/step - loss: 0.0834 - accuracy: 0.9750 - val_loss: 0.3579 - val_accuracy: 0.9125
Epoch 5/300
18/18 [==============================] - 5s 279ms/step - loss: 0.0134 - accuracy: 0.9964 - val_loss: 0.5394 - val_accuracy: 0.9062

wait mode, step: 1
Epoch 6/300
18/18 [==============================] - 5s 289ms/step - loss: 0.0046 - accuracy: 1.0000 - val_loss: 0.2931 - val_accuracy: 0.9125
Epoch 7/300
18/18 [==============================] - 5s 288ms/step - loss: 0.0019 - accuracy:

In [ ]:
%tensorboard --logdir "./logs"

In [ ]:
pred=np.argmax(model_dense.predict(X_test),axis=1)
print('accuracy of test for model with fix densenet weight==>%.4f'% accuracy_score(y_test,pred))
print('confusion matrix of test for model with fix densenet weight==>\n', confusion_matrix(y_test,pred))

accuracy of test for model with fix densenet weight==>0.8375
confusion matrix of test for model with fix densenet weight==>
 [[36  6]
 [ 7 31]]


#Fine tuning

In [ ]:
for layer in dense_based_model.layers[:-200]:
    layer.trainable = False
for layer in dense_based_model.layers[-200:]:
    layer.trainable = True

In [ ]:
model_dense_2=keras.models.Sequential()
model_dense_2.add(dense_based_model)
for i in range(1,len(model_dense.layers)):
  model_dense_2.add(model_dense.layers[i])

In [ ]:
%rm -rf './logs'
tb_callback=keras.callbacks.TensorBoard(log_dir="./logs",histogram_freq=1)

In [ ]:
model_dense_2.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss=keras.losses.SparseCategoricalCrossentropy(True),
              metrics=['accuracy'])
es_callback=EarlyStoppingCallback(patience=5)
model_dense_2.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=300, callbacks=(tb_callback,es_callback))

Epoch 1/300


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:4930: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


18/18 [==============================] - 19s 563ms/step - loss: 0.3033 - accuracy: 0.8679 - val_loss: 0.4209 - val_accuracy: 0.8750
Epoch 2/300
18/18 [==============================] - 5s 298ms/step - loss: 0.0727 - accuracy: 0.9750 - val_loss: 0.2332 - val_accuracy: 0.9062
Epoch 3/300
18/18 [==============================] - 5s 302ms/step - loss: 0.0377 - accuracy: 0.9911 - val_loss: 0.1994 - val_accuracy: 0.9062
Epoch 4/300
18/18 [==============================] - 5s 293ms/step - loss: 0.0246 - accuracy: 0.9982 - val_loss: 0.2421 - val_accuracy: 0.9125

wait mode, step: 1
Epoch 5/300
18/18 [==============================] - 5s 289ms/step - loss: 0.0142 - accuracy: 0.9982 - val_loss: 0.2342 - val_accuracy: 0.9062

wait mode, step: 2
Epoch 6/300
18/18 [==============================] - 5s 297ms/step - loss: 0.0115 - accuracy: 0.9982 - val_loss: 0.2816 - val_accuracy: 0.9000

wait mode, step: 3
Epoch 7/300
18/18 [==============================] - 5s 295ms/step - loss: 0.0137 - accuracy:

In [ ]:
%tensorboard --logdir "./logs"

In [ ]:
pred=np.argmax(model_dense_2.predict(X_test),axis=1)
print('accuracy of test for model with finetune densenet weight==>%.4f'% accuracy_score(y_test,pred))
print('confusion matrix of test for model with finetune densenet weight==>\n', confusion_matrix(y_test,pred))

accuracy of test for model with finetune densenet weight==>0.9375
confusion matrix of test for model with finetune densenet weight==>
 [[40  2]
 [ 3 35]]
